In [1]:
from pytube import YouTube
from youtube_transcript_api import YouTubeTranscriptApi
import ssl

class YouTubeDownloader:
    def __init__(self, video_url):
        self.video_url = video_url

    def download_video(self, output_path):
        try:
            # Disable SSL verification
            ssl._create_default_https_context = ssl._create_unverified_context
            yt = YouTube(self.video_url)
            stream = yt.streams.get_highest_resolution()
            stream.download(output_path)
            print(f"Video downloaded to {output_path}")
        except Exception as e:
            print(f"Error downloading video: {e}")

    def download_captions(self, output_path):
        try:
            # Disable SSL verification
            ssl._create_default_https_context = ssl._create_unverified_context
            srt = YouTubeTranscriptApi.get_transcript(self.video_url)
            with open(output_path, "w", encoding="utf-8") as f:
                for caption in srt:
                    f.write("{}\n".format(caption['text']))
            print(f"Captions downloaded to {output_path}")
        except Exception as e:
            print(f"Error downloading captions: {e}")

if __name__ == "__main__":
    video_url = "https://www.youtube.com/watch?v=DhmZ6W1UAv4"
    video_url2 = "DhmZ6W1UAv4"
    output_video_path = "Drone_Video1.mp4"
    output_captions_path = "FiveEyes_captions1.srt"

    downloader = YouTubeDownloader(video_url)
    downloader2 = YouTubeDownloader(video_url2)
    downloader.download_video(output_video_path)
    downloader2.download_captions(output_captions_path)


Video downloaded to Drone_Video1.mp4
Captions downloaded to FiveEyes_captions1.srt


In [14]:
import cv2
from filterpy.kalman import KalmanFilter
import numpy as np
import subprocess

# Function to detect drone position in a frame (using computer vision)
def detect_drone(frame):
    # Example: Using contour detection to find the drone position
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    _, threshold = cv2.threshold(gray_frame, 240, 255, cv2.THRESH_BINARY)
    
    contours, _ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:  # If contours are found
        contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(contour)
        drone_position = (x + w // 2, y + h // 2)  # Center of the bounding box
        
        # Draw rectangle around the detected drone (for visualization)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        return drone_position
    else:
        return None

# Initialize Kalman filter
kf = KalmanFilter(dim_x=4, dim_z=2)
kf.x = np.array([0, 0, 0, 0])  # Initial state: [x, y, dx/dt, dy/dt]

# Define measurement matrix (drone position)
kf.H = np.array([[1, 0, 0, 0],
                 [0, 1, 0, 0]])

# Define measurement uncertainty
kf.R = np.eye(2) * 0.01  # Adjust the uncertainty based on your scenario

# Define process uncertainty
kf.P *= 10

# Define transition matrix
dt = 1.0  # Time step (you may adjust this based on your video frame rate)
kf.F = np.array([[1, 0, dt, 0],
                 [0, 1, 0, dt],
                 [0, 0, 1, 0],
                 [0, 0, 0, 1]])

frames_with_drone = []
trajectory_points = []

# Process the video
video_path = 'Drone Tracking 1.mp4'  # Replace with your video path
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Detect drone position in the frame
    drone_pos = detect_drone(frame)
    
    # Perform Kalman filtering
    if drone_pos is not None:  # Ensure drone is detected
        measurement = np.array(drone_pos)
        kf.predict()
        kf.update(measurement)

        # Get filtered drone position
        filtered_drone_pos = kf.x[:2]

        # Store frame with the drone
        frames_with_drone.append(frame.copy())

        # Store drone position for trajectory plotting
        trajectory_points.append(tuple(map(int, filtered_drone_pos)))
        
        # Draw bounding box around the detected drone
        x, y = drone_pos
        cv2.rectangle(frame, (x - 20, y - 20), (x + 20, y + 20), (0, 255, 0), 2)

        # Display the frame with the filtered drone position and bounding box
        cv2.imshow('Drone Tracking', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# Create a new video from frames with the drone and trajectory
if frames_with_drone:
    # Draw trajectory on frames
    for i in range(1, len(trajectory_points)):
        cv2.line(frames_with_drone[i], trajectory_points[i - 1], trajectory_points[i], (255, 0, 0), 2)

    # Save frames with drone and trajectory as a new video
    output_video = 'drone1_trajectory.mp4'
    height, width, _ = frames_with_drone[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, 20.0, (width, height))

    for frame in frames_with_drone:
        out.write(frame)

    out.release()

    # Use FFmpeg to convert the output video to the desired format and compress
    subprocess.call(['ffmpeg', '-i', output_video, '-c:v', 'libx264', '-crf', '23', 'drone_output_1.mp4'])


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.3 (clang-1403.0.22.14.1)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disab

# Tracking Drone 2 - Video 2

In [4]:
from pytube import YouTube
from youtube_transcript_api import YouTubeTranscriptApi
import ssl

class YouTubeDownloader:
    def __init__(self, video_url):
        self.video_url = video_url

    def download_video(self, output_path):
        try:
            # Disable SSL verification
            ssl._create_default_https_context = ssl._create_unverified_context
            yt = YouTube(self.video_url)
            stream = yt.streams.get_highest_resolution()
            stream.download(output_path)
            print(f"Video downloaded to {output_path}")
        except Exception as e:
            print(f"Error downloading video: {e}")

if __name__ == "__main__":
    video_url = "https://www.youtube.com/watch?v=YrydHPwRelI&themeRefresh=1"
  
    output_video_path = "Drone_Video2.mp4"

    downloader = YouTubeDownloader(video_url)
  
    downloader.download_video(output_video_path)
  


Video downloaded to Drone_Video2.mp4


In [12]:
import cv2
from filterpy.kalman import KalmanFilter
import numpy as np
import subprocess

# Function to detect drone position in a frame (using computer vision)
def detect_drone(frame, background_subtractor):
    # Apply background subtraction
    fg_mask = background_subtractor.apply(frame)
    
    # Further processing (thresholding, morphology, contour detection)
    fg_mask = cv2.threshold(fg_mask, 240, 255, cv2.THRESH_BINARY)[1]
    fg_mask = cv2.erode(fg_mask, None, iterations=2)
    fg_mask = cv2.dilate(fg_mask, None, iterations=5)
    
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:
        contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(contour)
        drone_position = (x + w // 2, y + h // 2)  # Center of the bounding box
        
        # Draw rectangle around the detected drone (for visualization)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        return drone_position
    else:
        return None

# Prepare background subtractor
background_subtractor = cv2.createBackgroundSubtractorMOG2(history=100, detectShadows=False)

# Initialize Kalman filter
kf = KalmanFilter(dim_x=4, dim_z=2)
kf.x = np.array([0, 0, 0, 0])  # Initial state: [x, y, dx/dt, dy/dt]

# Define measurement matrix (drone position)
kf.H = np.array([[1, 0, 0, 0],
                 [0, 1, 0, 0]])

# Define measurement uncertainty
kf.R = np.eye(2) * 0.05  # Adjust the uncertainty based on your scenario

# Define process uncertainty
kf.P *= 10

# Define transition matrix
dt = 2.0  # Time step (you may adjust this based on your video frame rate)
kf.F = np.array([[1, 0, dt, 0],
                 [0, 1, 0, dt],
                 [0, 0, 1, 0],
                 [0, 0, 0, 1]])

frames_with_drone = []
trajectory_points = []

# Process the video
video_path = 'Drone Tracking 2.mp4'  # Replace with your video path
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Detect drone position in the frame
    drone_pos = detect_drone(frame, background_subtractor)
    
    # Perform Kalman filtering
    if drone_pos is not None:  # Ensure drone is detected
        measurement = np.array(drone_pos)
        kf.predict()
        kf.update(measurement)

        # Get filtered drone position
        filtered_drone_pos = kf.x[:2]

        # Store frame with the drone
        frames_with_drone.append(frame.copy())

        # Store drone position for trajectory plotting
        trajectory_points.append(tuple(map(int, filtered_drone_pos)))
        
        # Draw bounding box around the detected drone
        x, y = drone_pos
        cv2.rectangle(frame, (x - 20, y - 20), (x + 20, y + 20), (0, 255, 0), 2)

        # Display the frame with the filtered drone position and bounding box
        cv2.imshow('Drone Tracking', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# Create a new video from frames with the drone and trajectory
if frames_with_drone:
    # Draw trajectory on frames
    for i in range(1, len(trajectory_points)):
        cv2.line(frames_with_drone[i], trajectory_points[i - 1], trajectory_points[i], (255, 0, 0), 2)

    # Save frames with drone and trajectory as a new video
    output_video = 'drone2_trajectory.mp4'
    height, width, _ = frames_with_drone[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, 20.0, (width, height))

    for frame in frames_with_drone:
        out.write(frame)

    out.release()

    # Use FFmpeg to convert the output video to the desired format and compress
    subprocess.call(['ffmpeg', '-i', output_video, '-c:v', 'libx264', '-crf', '23', 'drone_output_2.mp4'])


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.3 (clang-1403.0.22.14.1)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disab